In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
print(os.listdir("../input"))

TRAIN_DIR = '../input/train-scene classification/train/'
TEST_DIR = '../input/test_WyRytb0.csv/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

y_test = pd.read_csv("../input/test_WyRytb0.csv")
y_train = pd.read_csv("../input/train-scene classification/train.csv")
y_train = y_train['label']
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, 6)

train_images_list = []
test_images_list = []
a = list(y_test['image_name'])
for i in train_images:
    if str(i[42:]) not in a:
        train_images_list.append(i)
    if str(i[42:]) in a:
        test_images_list.append(i)
        
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_images_list.sort(key=natural_keys)
test_images_list.sort(key=natural_keys)

def prepare_data(list_of_images):
    x = [] 
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (100,100), interpolation=cv2.INTER_CUBIC))     
    return x
X_train = prepare_data(train_images_list)
X_test = prepare_data(test_images_list)

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Sequential
from keras import optimizers
from keras import regularizers
model = Sequential()

model.add(Conv2D(64, (2, 2), input_shape = (100, 100, 3)))
model.add(Conv2D(64, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Conv2D(128, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.4))
model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.2))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.2))
model.add(Conv2D(256, (2, 2)))
model.add(keras.layers.LeakyReLU(alpha=0.4))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(.2))

model.add(Dense(6, activation = 'softmax'))

model.compile(optimizer=optimizers.Adam(lr=0.0001),loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_prime, X_val, y_train_prime, y_val = train_test_split(X_train, y_train_categorical, random_state = 6, train_size = .75)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow(np.array(X_train_prime), y_train_prime, batch_size = 32, shuffle = True)
validation_generator = val_datagen.flow(np.array(X_val), y_val, batch_size = 32, shuffle = True)


In [ ]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch = 1000,
    epochs = 100,
    shuffle = True,
    validation_data = validation_generator,
    validation_steps = 400, use_multiprocessing=True
)

In [ ]:
X_test = np.array(X_test)/255.0
prediction_probabilities = model.predict(X_test)
counter = range(1, len(test_images_list) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']
for i, j in enumerate(solution['label']):
    print(i+1)
    solution['label'][i] = np.argmax(j)
solution.to_csv("SceneClassification5.csv", index = False)